In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import wandb
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, Subset, DataLoader, Dataset

In [ ]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")

device = 'mps' # 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
checkpoints_path = '/Users/sienkadounia/lab/ai-futures/Project/ewdd/'
label_noise_path = '/Users/sienkadounia/lab/ai-futures/Project/label_noise/'
rlcts_path = '/Users/sienkadounia/lab/checkpoints/rlcts/ewdd/'
mwdd_path = '/Users/sienkadounia/lab/ai-futures/Project/mwdd'

In [ ]:
batch_size = 128
epochs = 2000
num_classes = 10
lr = 0.1
use_label_noise =False
augmented = False
use_adam_op = False

In [ ]:
# Load and preprocess the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
mnist_train = datasets.MNIST('data/',
                             train=True,
                             download=True,
                             transform=transform)
mnist_test = datasets.MNIST('data/',
                            train=False,
                            download=True,
                            transform=transform)

# Randomly select 40k samples from the training set
indices = torch.randperm(len(mnist_train))[:40000]
train_data = torch.utils.data.Subset(mnist_train, indices)

train_loader = DataLoader(train_data,
                          batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader(mnist_test,
                        batch_size = batch_size,
                        shuffle =True)

In [ ]:
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
def initialize_with_weight_reuse(larger_model, smaller_model, additional_hidden_size):
    # Copy weights for the first part of the network
    larger_model.fc1.weight.data[:smaller_model.fc1.weight.size(0)] = smaller_model.fc1.weight.data
    larger_model.fc1.bias.data[:smaller_model.fc1.bias.size(0)] = smaller_model.fc1.bias.data
    
    # Initialize additional weights normally
    nn.init.normal_(larger_model.fc1.weight.data[smaller_model.fc1.weight.size(0):], mean=0, std=0.01)
    nn.init.zeros_(larger_model.fc1.bias.data[smaller_model.fc1.bias.size(0):])
    
    # Initialize output layer weights with Glorot-uniform
    nn.init.xavier_uniform_(larger_model.fc2.weight)
    nn.init.zeros_(larger_model.fc2.bias)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
from torch.optim import lr_scheduler
import math
def lr_lambda(epoch):
    base_lr = 0.1
    return base_lr/math.sqrt(epoch+1)

In [ ]:
def train_network(model, train_loader, input_size, hidden_size, output_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.95) if not use_adam_op else optim.Adam(model.parameters(), lr=lr)
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

    wandb.init(project="SLT of Double Descent", 
    config = {'epochs': epochs,
              'classes': num_classes,
              'learning_rate': lr,
              'use_label_noise': use_label_noise,
              'dataset': "MNIST",
              'architecture': "FCNN",
              'model_width': count_parameters(model),
              'augmented': augmented,
              'adam optimizer': use_adam_op})

    wandb.watch(model)
    config = wandb.config

    test_losses = []
    train_losses = []
    train_accs = []
    test_accs = []
    step = 0
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        train_loss = 0.0
        train_acc = 0.0
        for _, data in enumerate(train_loader):
            images, labels = data
            images = images.view(-1, input_size)
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            batch_loss = criterion(outputs, labels)
            predictions = outputs.argmax(axis=-1)
            train_acc += torch.sum(predictions == labels).item()
            train_loss += batch_loss.item()
            batch_loss.backward()
            optimizer.step()
            step += 1

            wandb.log({'batch_loss': batch_loss.item()}, step=step)

        # Calculate average training loss for the epoch
        train_loss /= len(train_loader)
        train_acc /= 40000
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation (or test) loop
        model.eval()

        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for k, test_data in enumerate(test_loader):
                test_images, test_labels = test_data
                test_images = test_images.view(-1, input_size)
                test_images = test_images.to(device)
                test_labels = test_labels.to(device)

                outputs = model(test_images)
                batch_test_loss = criterion(outputs, test_labels)
                test_loss += batch_test_loss.item()

                predictions = outputs.argmax(axis=-1)
                test_acc += torch.sum(predictions == test_labels).item()

        # Let's calculate average test loss for the epoch
        test_loss /= len(test_loader)
        test_acc /= len(test_loader.dataset.data)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        torch.save(test_accs, rlcts_path + 'test_accs_'+ str(hidden_size)+'.pt')
        torch.save(test_losses, rlcts_path + 'test_losses_'+ str(hidden_size)+'.pt')

        wandb.log({'epoch': epoch,
                   'loss/train': train_loss,
                   'loss/test': test_loss,
                   'accuracy/train': train_acc,
                   'accuracy/test': test_acc
                   }, step=step)

        # Print or log the training and test losses for each epoch
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

        scheduler.step()
        
        # Implement stopping condition based on classification error or epoch limit
        # Note: Simplified, as the full stopping condition is not detailed here
    wandb.finish()
    torch.save(model.state_dict(), mwdd_path + 'fccnn_'+str(hidden_size)+'.pth')
    print(f"Training completed for hidden size: {hidden_size}")

In [ ]:
input_size = 28 * 28  # MNIST images are 28x28
output_size = 10  # 10 classes
hidden_sizes = [2, 4, 8, 16, 32 ,64, 128]  # Example sizes

previous_model = None
for k in range (len(hidden_sizes)):
    hidden_size = hidden_sizes[k]
    model = FCNN(input_size, hidden_size, output_size).to(device)
    '''if previous_model:
        initialize_with_weight_reuse(model, previous_model, hidden_size)
    print('Started traing for model')
    train_network(model, train_loader, input_size, hidden_size, output_size)
    previous_model = model'''
    train_network(model, train_loader, input_size, hidden_size, output_size)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sienka. Use `wandb login --relogin` to force relogin


Epoch 1/2000, Train Loss: 1.9905, Test Loss: 1.8072
Epoch 2/2000, Train Loss: 1.7684, Test Loss: 1.7422
Epoch 3/2000, Train Loss: 1.7239, Test Loss: 1.6965
Epoch 4/2000, Train Loss: 1.6987, Test Loss: 1.6853
Epoch 5/2000, Train Loss: 1.6882, Test Loss: 1.6775
Epoch 6/2000, Train Loss: 1.6825, Test Loss: 1.6703
Epoch 7/2000, Train Loss: 1.6695, Test Loss: 1.6657
Epoch 8/2000, Train Loss: 1.6677, Test Loss: 1.6609
Epoch 9/2000, Train Loss: 1.6630, Test Loss: 1.6535
Epoch 10/2000, Train Loss: 1.6609, Test Loss: 1.6555
Epoch 11/2000, Train Loss: 1.6583, Test Loss: 1.6490
Epoch 12/2000, Train Loss: 1.6549, Test Loss: 1.6537
Epoch 13/2000, Train Loss: 1.6553, Test Loss: 1.6475
Epoch 14/2000, Train Loss: 1.6514, Test Loss: 1.6385
Epoch 15/2000, Train Loss: 1.6510, Test Loss: 1.6411
Epoch 16/2000, Train Loss: 1.6478, Test Loss: 1.6408
Epoch 17/2000, Train Loss: 1.6487, Test Loss: 1.6323
Epoch 18/2000, Train Loss: 1.6468, Test Loss: 1.6487
Epoch 19/2000, Train Loss: 1.6453, Test Loss: 1.6512
Ep

wandb: Network error (ConnectTimeout), entering retry loop.


Epoch 851/2000, Train Loss: 1.5276, Test Loss: 1.5205
Epoch 852/2000, Train Loss: 1.5279, Test Loss: 1.5244
Epoch 853/2000, Train Loss: 1.5281, Test Loss: 1.5316
Epoch 854/2000, Train Loss: 1.5276, Test Loss: 1.5210
Epoch 855/2000, Train Loss: 1.5277, Test Loss: 1.5322
Epoch 856/2000, Train Loss: 1.5281, Test Loss: 1.5244
Epoch 857/2000, Train Loss: 1.5280, Test Loss: 1.5307
Epoch 858/2000, Train Loss: 1.5280, Test Loss: 1.5234
Epoch 859/2000, Train Loss: 1.5278, Test Loss: 1.5249
Epoch 860/2000, Train Loss: 1.5282, Test Loss: 1.5262
Epoch 861/2000, Train Loss: 1.5285, Test Loss: 1.5291
Epoch 862/2000, Train Loss: 1.5290, Test Loss: 1.5273
Epoch 863/2000, Train Loss: 1.5279, Test Loss: 1.5496
Epoch 864/2000, Train Loss: 1.5280, Test Loss: 1.5232
Epoch 865/2000, Train Loss: 1.5278, Test Loss: 1.5294
Epoch 866/2000, Train Loss: 1.5277, Test Loss: 1.5220
Epoch 867/2000, Train Loss: 1.5279, Test Loss: 1.5227
Epoch 868/2000, Train Loss: 1.5281, Test Loss: 1.5288
Epoch 869/2000, Train Loss: 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy/test,▁▇██▇▇█▆▇▇▇▇▇▆▇▇▇▆▇▆▇▇▆▆▆▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆
accuracy/train,▁▆▇█████████████████████████████████████
batch_loss,█▇▄▃▂▂▂▂▃▂▂▂▁▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/test,▂▁▁▁▂▃▃▃▄▅▄▄▄▅▆▅▅▅▅▆▅▆▆▇▇▆▆▆█▆▇▆▆▆▇▇▇▇▇▇
loss/train,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy/test,0.9703
accuracy/train,1.0
batch_loss,0.00152
epoch,1999
loss/test,0.13928


Training completed for hidden size: 64


Epoch 1/2000, Train Loss: 0.5117, Test Loss: 0.2641
Epoch 2/2000, Train Loss: 0.2314, Test Loss: 0.1935
Epoch 3/2000, Train Loss: 0.1759, Test Loss: 0.1742
Epoch 4/2000, Train Loss: 0.1506, Test Loss: 0.1458
Epoch 5/2000, Train Loss: 0.1300, Test Loss: 0.1374
Epoch 6/2000, Train Loss: 0.1183, Test Loss: 0.1379
Epoch 7/2000, Train Loss: 0.1089, Test Loss: 0.1259
Epoch 8/2000, Train Loss: 0.1009, Test Loss: 0.1148
Epoch 9/2000, Train Loss: 0.0928, Test Loss: 0.1114
Epoch 10/2000, Train Loss: 0.0872, Test Loss: 0.1114
Epoch 11/2000, Train Loss: 0.0825, Test Loss: 0.1055
Epoch 12/2000, Train Loss: 0.0769, Test Loss: 0.1029
Epoch 13/2000, Train Loss: 0.0731, Test Loss: 0.1014
Epoch 14/2000, Train Loss: 0.0700, Test Loss: 0.0980
Epoch 15/2000, Train Loss: 0.0667, Test Loss: 0.0979
Epoch 16/2000, Train Loss: 0.0630, Test Loss: 0.0953
Epoch 17/2000, Train Loss: 0.0611, Test Loss: 0.0977
Epoch 18/2000, Train Loss: 0.0591, Test Loss: 0.0953
Epoch 19/2000, Train Loss: 0.0556, Test Loss: 0.0957
Ep

KeyboardInterrupt: 

### Fixed learning rate, 4000 epochs

In [ ]:
epochs = 400
lr = 0.002
use_adam_op = True
def train_network(model, train_loader, input_size, hidden_size, output_size):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.95) if not use_adam_op else optim.Adam(model.parameters(), lr=lr)
    #scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

    wandb.init(project="SLT of Double Descent", 
    config = {'epochs': epochs,
              'classes': num_classes,
              'learning_rate': lr,
              'use_label_noise': use_label_noise,
              'dataset': "MNIST",
              'architecture': "FCNN",
              'model_width': count_parameters(model),
              'augmented': augmented,
              'adam optimizer': use_adam_op})

    wandb.watch(model)
    config = wandb.config

    test_losses = []
    train_losses = []
    train_accs = []
    test_accs = []
    step = 0
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        train_loss = 0.0
        train_acc = 0.0
        for _, data in enumerate(train_loader):
            images, labels = data
            images = images.view(-1, input_size)
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            batch_loss = criterion(outputs, labels)
            predictions = outputs.argmax(axis=-1)
            train_acc += torch.sum(predictions == labels).item()
            train_loss += batch_loss.item()
            batch_loss.backward()
            optimizer.step()
            step += 1

            wandb.log({'batch_loss': batch_loss.item()}, step=step)

        # Calculate average training loss for the epoch
        train_loss /= len(train_loader)
        train_acc /= 40000
        train_losses.append(train_loss)
        train_accs.append(train_acc)

        # Validation (or test) loop
        model.eval()

        test_loss = 0.0
        test_acc = 0.0
        with torch.no_grad():
            for k, test_data in enumerate(test_loader):
                test_images, test_labels = test_data
                test_images = test_images.view(-1, input_size)
                test_images = test_images.to(device)
                test_labels = test_labels.to(device)

                outputs = model(test_images)
                batch_test_loss = criterion(outputs, test_labels)
                test_loss += batch_test_loss.item()

                predictions = outputs.argmax(axis=-1)
                test_acc += torch.sum(predictions == test_labels).item()

        # Let's calculate average test loss for the epoch
        test_loss /= len(test_loader)
        test_acc /= len(test_loader.dataset.data)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        torch.save(test_accs, rlcts_path + 'test_accs_'+ str(hidden_size)+'.pt')
        torch.save(test_losses, rlcts_path + 'test_losses_'+ str(hidden_size)+'.pt')

        wandb.log({'epoch': epoch,
                   'loss/train': train_loss,
                   'loss/test': test_loss,
                   'accuracy/train': train_acc,
                   'accuracy/test': test_acc
                   }, step=step)

        # Print or log the training and test losses for each epoch
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

        #scheduler.step()
        
        # Implement stopping condition based on classification error or epoch limit
        # Note: Simplified, as the full stopping condition is not detailed here
    wandb.finish()
    torch.save(model.state_dict(), mwdd_path + 'fccnn_'+str(hidden_size)+'_fixed.pth')
    print(f"Training completed for hidden size: {hidden_size}")

In [ ]:
input_size = 28 * 28  # MNIST images are 28x28
output_size = 10  # 10 classes
hidden_sizes = [2, 4, 8, 16, 32 ,64, 128, 512]  # Example sizes

previous_model = None
for k in range (len(hidden_sizes)):
    hidden_size = hidden_sizes[k]
    torch.manual_seed(42)
    model = FCNN(input_size, hidden_size, output_size).to(device)
    '''if previous_model:
        initialize_with_weight_reuse(model, previous_model, hidden_size)
    print('Started traing for model')
    train_network(model, train_loader, input_size, hidden_size, output_size)
    previous_model = model'''
    train_network(model, train_loader, input_size, hidden_size, output_size)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy/test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy/train,▁███████████████████████████████████████
batch_loss,▂▅▅▁▃▇▄▂▄▄▃▃▃▄▂▂▄▅▇▅▂▄▂▄▂▁▂▃▃▅▆▄▃▃▁▅▇▅█▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/test,▅▄▆▆▂▂▃▅▄▄▄▇▅▃▃▁▆▅▄▄▄▄▅▁▃▆▂▅▂▃▃▅▄▄█▃▆▂▄▄
loss/train,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy/test,0.1135
accuracy/train,0.11238
batch_loss,2.30383
epoch,77
loss/test,2.30107


Epoch 1/400, Train Loss: 2.3235, Test Loss: 2.3033
Epoch 2/400, Train Loss: 2.3021, Test Loss: 2.3010
Epoch 3/400, Train Loss: 2.3013, Test Loss: 2.3005
Epoch 4/400, Train Loss: 2.3012, Test Loss: 2.3012
Epoch 5/400, Train Loss: 2.3013, Test Loss: 2.3011
Epoch 6/400, Train Loss: 2.3013, Test Loss: 2.3011
Epoch 7/400, Train Loss: 2.3013, Test Loss: 2.3010
Epoch 8/400, Train Loss: 2.3013, Test Loss: 2.3008
Epoch 9/400, Train Loss: 2.3013, Test Loss: 2.3011
Epoch 10/400, Train Loss: 2.3013, Test Loss: 2.3008
Epoch 11/400, Train Loss: 2.3013, Test Loss: 2.3009
Epoch 12/400, Train Loss: 2.3013, Test Loss: 2.3009
Epoch 13/400, Train Loss: 2.3013, Test Loss: 2.3010
Epoch 14/400, Train Loss: 2.3013, Test Loss: 2.3012
Epoch 15/400, Train Loss: 2.3012, Test Loss: 2.3009
Epoch 16/400, Train Loss: 2.3013, Test Loss: 2.3010
Epoch 17/400, Train Loss: 2.3012, Test Loss: 2.3011
Epoch 18/400, Train Loss: 2.3013, Test Loss: 2.3010
Epoch 19/400, Train Loss: 2.3013, Test Loss: 2.3010
Epoch 20/400, Train L

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy/test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy/train,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_loss,▅▇▅▂▅▁▁▃▂▇▃▇▇▃▂▅▅▄▇█▃▄▆▄▃▅▄▃▄▇▄▃▅▄▄▆▁▃▆▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/test,▄▂▄▅▄▂▅▄▄█▄▂▂▂▂█▅▂▄▂▅█▄▃▄▃▃▄▂▃▃▅▅▆▃▃▂▅▁▃
loss/train,▁▄▄▆▄▂▄▂▅▄▆▇▅▃▆▅▅▆▇█▄▄▃▅▆▃▅▇▆▆▄▆▇▄▆▅▆▅▅▄
accuracy/test,0.1135
accuracy/train,0.11238
batch_loss,2.3041
epoch,399
loss/test,2.30098


...

wandb: Network error (ConnectionError), entering retry loop.


Epoch 344/400, Train Loss: 0.0000, Test Loss: 0.5579
Epoch 345/400, Train Loss: 0.0000, Test Loss: 0.5576
Epoch 346/400, Train Loss: 0.0000, Test Loss: 0.5574
Epoch 347/400, Train Loss: 0.0000, Test Loss: 0.5571
Epoch 348/400, Train Loss: 0.0000, Test Loss: 0.5569
Epoch 349/400, Train Loss: 0.0000, Test Loss: 0.5567
Epoch 350/400, Train Loss: 0.0000, Test Loss: 0.5564
Epoch 351/400, Train Loss: 0.0000, Test Loss: 0.5731
Epoch 352/400, Train Loss: 0.0000, Test Loss: 0.5559
Epoch 353/400, Train Loss: 0.0000, Test Loss: 0.5557
Epoch 354/400, Train Loss: 0.0000, Test Loss: 0.5554
Epoch 355/400, Train Loss: 0.0000, Test Loss: 0.5552
Epoch 356/400, Train Loss: 0.0000, Test Loss: 0.5550
Epoch 357/400, Train Loss: 0.0000, Test Loss: 0.5547
Epoch 358/400, Train Loss: 0.0000, Test Loss: 0.5545
Epoch 359/400, Train Loss: 0.0000, Test Loss: 0.5542
Epoch 360/400, Train Loss: 0.0000, Test Loss: 0.5569
Epoch 361/400, Train Loss: 0.0000, Test Loss: 0.5537
Epoch 362/400, Train Loss: 0.0000, Test Loss: 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy/test,▁▃▃▅▅▇▄▆▆▆▆▆▅▅▆▅▅▅▆▆▆▅▇▆▆▆▇▇▆█▆▆████████
accuracy/train,▁▅▆▆▇▇▇▇▇▇▇▇▇▇████████████▇█████████████
batch_loss,█▂▃▁▂▁▁▁▁▁▁▁▁▁▁▂▁▂▆▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/test,▁▁▁▁▂▂▂▃▂▃▄▄▄▅▄▅▄▅▅▅▅▆▅▆▆▆▆▇▇▇▇█▆▆▆▆▆▆▆▆
loss/train,█▄▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▃▂▂▁▂▁▁▁▁▁▁▁▁▁
accuracy/test,0.982
accuracy/train,1.0
batch_loss,0.0
epoch,399
loss/test,0.55091


Training completed for hidden size: 512
